In [9]:
import feedparser
import json
import requests
from datetime import datetime

# Fonction pour convertir une date Feedparser en ISO 8601
def convert_to_iso8601(date_str):
    try:
        # Parse la date avec le format de Feedparser
        parsed_date = datetime.strptime(date_str, "%a, %d %b %Y %H:%M:%S %z")
        # Convertit en format ISO 8601
        return parsed_date.isoformat()
    except ValueError:
        return None  # Retourne None si la date est invalide ou absente


# Liste des flux RSS (Feedly et Google Alerts)
rss_feeds = [
    "http://javascript.developpez.com/index/rss",  # lien feedly
    "https://www.google.com/alerts/feeds/11440117351863688430/4739719902568766787",     # Google Alert
]

# Notion API Configuration
NOTION_API_URL = "https://api.notion.com/v1/pages"
NOTION_API_KEY = "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"  # Remplace par ton Token Notion
DATABASE_ID = "144e6de0986180ada0b3c86d313465c2"  # Remplace par l'ID de ta base de données

# Fonction pour analyser les flux RSS
def parse_rss_feeds(feeds):
    articles = []
    for feed_url in feeds:
        print(f"Analyse du flux RSS : {feed_url}")
        feed = feedparser.parse(feed_url)
        
        # Vérifie si le flux est valide
        if 'title' in feed.feed:
            print(f"Flux trouvé : {feed.feed.title}")
        else:
            print("Erreur : Flux non valide.")
            continue
        
        for entry in feed.entries:
            article = {
                "title": entry.title,
                "link": entry.link,
                "published": entry.published if "published" in entry else "N/A",
                "summary": entry.summary if "summary" in entry else "N/A"
            }
            articles.append(article)
    
    return articles

# Fonction pour envoyer un article à Notion
def add_article_to_notion(article):
    headers = {
        "Authorization": f"Bearer {NOTION_API_KEY}",
        "Content-Type": "application/json",
        "Notion-Version": "2022-06-28"
    }
    # Conversion de la date
    iso_date = convert_to_iso8601(article["published"])

    data = {
        "parent": {"database_id": DATABASE_ID},
        "properties": {
            "Title": {"title": [{"text": {"content": article["title"]}}]},
            "Link": {"url": article["link"]},
            "Published": {"date": {"start": iso_date} if iso_date else None},
            "Summary": {"rich_text": [{"text": {"content": article["summary"]}}]}
        }
    }
    response = requests.post(NOTION_API_URL, headers=headers, json=data)
    if response.status_code == 200:
        print(f"Article ajouté à Notion : {article['title']}")
    else:
        print(f"Erreur lors de l'ajout à Notion : {response.status_code}, {response.text}")


# Fonction principale
if __name__ == "__main__":
    # Étape 1 : Analyse des flux RSS
    collected_articles = parse_rss_feeds(rss_feeds)
    
    # Étape 3 : Envoi à Notion
    print(f"\nAjout des articles à Notion...")
    for article in collected_articles:
        add_article_to_notion(article)

    print(f"\n{len(collected_articles)} articles ajoutés à Notion.")


Analyse du flux RSS : http://javascript.developpez.com/index/rss
Flux trouvé : Flux JavaScript Developpez
Analyse du flux RSS : https://www.google.com/alerts/feeds/11440117351863688430/4739719902568766787
Flux trouvé : Google Alert - javascript

Ajout des articles à Notion...
Article ajouté à Notion : WebStorm 2024.3 est disponible :  prise en charge intégrée des outils de base de données et de SQL, saisie semi-automatique de code générée par IA, et bien plus
Article ajouté à Notion : Invasion silencieuse : des centaines de bibliothèques malveillantes publiées sur NPM tentent d'installer des malwares sur les machines des développeurs pour infiltrer les entreprises
Article ajouté à Notion : Claude, le chatbot IA d'Anthropic, peut désormais écrire et exécuter du code JavaScript. L'entreprise a lancé un nouvel outil d'analyse qui permet à Claude d'analyser des ensembles de données divers
Article ajouté à Notion : Les EDI de JetBrains WebStorm et Rider sont désormais gratuits pour une util